## Fase 5 — EDA e Pré-processamento

Nesta fase iniciaremos o **Item 5 do Plano de Atividades**, responsável por abrir a etapa de **Análise Exploratória de Dados (EDA)** do conjunto real de dados versionado.  
O objetivo é garantir que os arquivos `train.csv` e `test.csv` estejam corretamente estruturados, analisar tipos de variáveis, valores ausentes, estatísticas básicas e possíveis inconsistências.  
Todo o trabalho será executado dentro do **DevContainer**, com **rastreabilidade total**, utilizando sempre caminhos relativos revisados fisicamente, em conformidade com o **PROTOCOLO V5.4**.


In [2]:
# ETAPA: CARREGAR E INSPECIONAR O DATASET REAL

import os
import pandas as pd

# Confirma diretório de trabalho do Kernel
print("CWD Kernel:", os.getcwd())

# Define caminho coerente para o dataset real
# Se o notebook estiver dentro de notebooks/, usar ../data/raw/
data_raw_dir = "../data/raw"

# Lista arquivos para validar nomes reais
print("Conteúdo de data/raw/:", os.listdir(data_raw_dir))

# Carrega arquivos reais versionados
df_train = pd.read_csv(os.path.join(data_raw_dir, "train.csv"))
df_test = pd.read_csv(os.path.join(data_raw_dir, "test.csv"))

# Verifica estrutura inicial
print("Formato treino:", df_train.shape)
print("Formato teste:", df_test.shape)

print("\nTipos de colunas treino:")
print(df_train.dtypes)

print("\nValores nulos treino:")
print(df_train.isnull().sum())

print("\nAmostra treino:")
print(df_train.head(20))


CWD Kernel: /workspace/notebooks
Conteúdo de data/raw/: ['test.csv', 'train.csv']


/tmp/ipykernel_310944/465301536.py:17: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(os.path.join(data_raw_dir, "train.csv"))


Formato treino: (100000, 28)
Formato teste: (50000, 27)

Tipos de colunas treino:
ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour 

## Análise da Célula de Carregamento e Inspeção Inicial

A leitura dos arquivos `train.csv` e `test.csv` confirma que o dataset real está estruturado e coerente com o versionamento em `data/raw/`.  
O `train.csv` possui **100.000 linhas × 28 colunas** e o `test.csv` **50.000 linhas × 27 colunas**. A listagem do diretório validou os nomes dos arquivos sem dependência de heurística, alinhado ao **PROTOCOLO V5.4**.

### Principais observações:

- **DtypeWarning:** A leitura acusou tipos mistos em algumas colunas (ex.: coluna 26), indicando registros com strings, símbolos ou formatos inconsistentes em campos que deveriam ser numéricos.
- **Tipos de colunas:** Diversas variáveis numéricas (`Age`, `Outstanding_Debt`, `Amount_invested_monthly` etc.) foram lidas como `object`. Isso confirma a necessidade de conversão explícita para tipos numéricos (`int64` ou `float64`) usando coerção.
- **Valores ausentes:** A contagem de `NaN` revelou lacunas relevantes em campos como `Name` (~10%), `Monthly_Inhand_Salary` (~15%), `Credit_History_Age` (~9%) e `Num_of_Delayed_Payment`. Será necessário definir políticas de imputação ou descarte.
- **Amostra de registros:** Identificou anomalias claras, como `Age` negativo ou inválido (`-500`, `28_`), ocupações `_______` e padrões de ruído como `!@9#%8` em `Payment_Behaviour`. Estes valores precisarão ser tratados na fase de limpeza.

Esta análise serve como base para planejar o pré-processamento, garantindo integridade e consistência antes de avançar para treinamento e rastreio de experimentos.


---

##  Políticas de Tratamento de Dados — Definição Oficial

Com base na análise exploratória inicial e nos requisitos do exercício, definimos as seguintes diretrizes de limpeza e padronização para o dataset `credit-score-classification`. Todas as etapas respeitam o **PROTOCOLO V5.4** e garantem rastreabilidade do pipeline.

### 🔹 Conversão de Tipos
- Todas as colunas identificadas como `object` mas que representam valores numéricos (ex.: `Age`, `Outstanding_Debt`, `Amount_invested_monthly`) serão convertidas para `float64` usando `pd.to_numeric(errors='coerce')`.  
- Valores que não puderem ser convertidos se tornarão `NaN` de forma rastreável.

### 🔹 Tratamento de Anomalias Numéricas
- Idades negativas ou entradas não numéricas (ex.: `-500`, `28_`) serão convertidas em `NaN` e imputadas com a mediana da coluna `Age`.
- Valores absurdos ou placeholders (ex.: `__10000__`) terão caracteres especiais removidos e serão convertidos para número se possível.

### 🔹 Valores Ausentes
- Variáveis numéricas contínuas com nulos (ex.: `Monthly_Inhand_Salary`, `Credit_History_Age`) serão imputadas com a mediana, pois são menos sensíveis a outliers.
- Variáveis categóricas com nulos (ex.: `Occupation`, `Payment_Behaviour`) receberão a categoria `Unknown` para manter integridade sem supor informação.

### 🔹 Outliers
- Para colunas financeiras como `Annual_Income` e `Outstanding_Debt`, aplicaremos Winsorization limitando os extremos aos percentis 1% e 99% para reduzir impacto de valores distorcidos.

### 🔹 Padronização de Strings
- Placeholder como `_______` e ruídos do tipo `!@9#%8` serão substituídos por `Unknown`.
- Campos com `Yes/No` serão convertidos para padrão binário se necessário na fase de modelagem.

### 🔹 Split e Persistência
- Após o tratamento, o dataset limpo será salvo em `data/processed/` versionado com DVC, pronto para rastreio no MLflow.
- Esta etapa encerra o pré-processamento, mantendo coerência para o treino e serving via API posteriormente.


In [3]:
# ETAPA: CONVERSÃO DE TIPOS NUMÉRICOS COM COERÇÃO

import pandas as pd

# Lista de colunas identificadas como numéricas, mas com tipo object
numeric_cols = [
    "Age",
    "Annual_Income",
    "Outstanding_Debt",
    "Amount_invested_monthly",
    "Monthly_Balance"
]

# Antes: tipos originais
print("Tipos originais:")
print(df_train[numeric_cols].dtypes)

# Aplica coerção para float64
for col in numeric_cols:
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')

# Depois: tipos convertidos
print("\nTipos após coerção:")
print(df_train[numeric_cols].dtypes)

# Verifica quantos valores viraram NaN
print("\nValores NaN adicionados por coerção:")
print(df_train[numeric_cols].isnull().sum())


Tipos originais:
Age                        object
Annual_Income              object
Outstanding_Debt           object
Amount_invested_monthly    object
Monthly_Balance            object
dtype: object

Tipos após coerção:
Age                        float64
Annual_Income              float64
Outstanding_Debt           float64
Amount_invested_monthly    float64
Monthly_Balance            float64
dtype: object

Valores NaN adicionados por coerção:
Age                        4939
Annual_Income              6980
Outstanding_Debt           1009
Amount_invested_monthly    8784
Monthly_Balance            1209
dtype: int64


## Imputação de Valores Ausentes — Colunas Numéricas

Após converter colunas numéricas para tipos coerentes, aplicamos imputação de `NaN` usando **mediana** para variáveis contínuas.  
Esta abordagem é robusta contra distorções de valores extremos, mantém o viés controlado e respeita o histórico real do dataset.



In [4]:
# ETAPA: IMPUTAR VALORES AUSENTES COM MEDIANA

# Define novamente as colunas numéricas que já foram convertidas
numeric_cols = [
    "Age",
    "Annual_Income",
    "Outstanding_Debt",
    "Amount_invested_monthly",
    "Monthly_Balance"
]

# Antes: mostra quantos NaN existem
print("NaN antes da imputação:")
print(df_train[numeric_cols].isnull().sum())

# Aplica imputação da mediana, coluna por coluna
for col in numeric_cols:
    median_value = df_train[col].median()
    df_train[col] = df_train[col].fillna(median_value)
    print(f"Imputado {col} com mediana = {median_value}")

# Depois: verifica se ainda restaram NaN
print("\nNaN após imputação:")
print(df_train[numeric_cols].isnull().sum())


NaN antes da imputação:
Age                        4939
Annual_Income              6980
Outstanding_Debt           1009
Amount_invested_monthly    8784
Monthly_Balance            1209
dtype: int64
Imputado Age com mediana = 33.0
Imputado Annual_Income com mediana = 37550.74
Imputado Outstanding_Debt com mediana = 1166.37
Imputado Amount_invested_monthly com mediana = 128.95453805190283
Imputado Monthly_Balance com mediana = 336.73122455696387

NaN após imputação:
Age                        0
Annual_Income              0
Outstanding_Debt           0
Amount_invested_monthly    0
Monthly_Balance            0
dtype: int64


## Limpeza de Placeholders e Ruídos — Colunas Categóricas

Após a imputação numérica, iniciamos a padronização de colunas categóricas que apresentam valores placeholders ou símbolos de ruído.  
Entradas como `_______`, `______`, `__`, `!@9#%8` e variações similares serão substituídas por `Unknown` para garantir consistência e coerência nos modelos.  
Esta abordagem segue o princípio de **não supor valores** quando não há base para inferência.


In [5]:
# ETAPA: LIMPEZA DE PLACEHOLDERS E RUÍDOS EM CATEGÓRICAS

# Lista de colunas categóricas sensíveis a ruído (ajuste conforme necessário)
categorical_cols = [
    "Occupation",
    "Payment_Behaviour",
    "Credit_Mix"
]

# Padrões que vamos considerar como ruído ou placeholder
placeholder_patterns = ["_______", "______", "__", "!@9#%8", "_", "__10000__"]

# Substitui por 'Unknown'
for col in categorical_cols:
    original_unique = df_train[col].unique()
    df_train[col] = df_train[col].replace(placeholder_patterns, "Unknown")
    updated_unique = df_train[col].unique()
    print(f"\nColuna: {col}")
    print(f"Antes: {original_unique}")
    print(f"Depois: {updated_unique}")



Coluna: Occupation
Antes: ['Scientist' '_______' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer'
 'Lawyer' 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant'
 'Musician' 'Mechanic' 'Writer' 'Architect']
Depois: ['Scientist' 'Unknown' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer'
 'Lawyer' 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant'
 'Musician' 'Mechanic' 'Writer' 'Architect']

Coluna: Payment_Behaviour
Antes: ['High_spent_Small_value_payments' 'Low_spent_Large_value_payments'
 'Low_spent_Medium_value_payments' 'Low_spent_Small_value_payments'
 'High_spent_Medium_value_payments' '!@9#%8'
 'High_spent_Large_value_payments']
Depois: ['High_spent_Small_value_payments' 'Low_spent_Large_value_payments'
 'Low_spent_Medium_value_payments' 'Low_spent_Small_value_payments'
 'High_spent_Medium_value_payments' 'Unknown'
 'High_spent_Large_value_payments']

Coluna: Credit_Mix
Antes: ['_' 'Good' 'Standard' 'Bad']
Depois: ['Unknown' 'Good' 'Standard' 'Bad']


## Persistência do Dataset Limpo em `data/processed/`

Após conversão de tipos, imputação de valores ausentes e padronização de placeholders, o dataset está pronto para ser salvo em `data/processed/`.  
A pasta segue a estrutura **cookiecutter-data-science**, separando dados brutos (`raw/`) de dados prontos para modelagem (`processed/`).  
O artefato será versionado com DVC para garantir rastreabilidade integral até o MLflow.


In [6]:
# ETAPA: SALVAR DATASET TRATADO EM data/processed/

import os

# Define caminho coerente
processed_dir = "../data/processed"  # se seu notebook roda em /notebooks
os.makedirs(processed_dir, exist_ok=True)

# Nome do arquivo tratado
output_path = os.path.join(processed_dir, "train_clean.csv")

# Salva CSV limpo
df_train.to_csv(output_path, index=False)

print(f"Dataset salvo em: {output_path}")


Dataset salvo em: ../data/processed/train_clean.csv


In [7]:
!dvc add ../data/processed/train_clean.csv
!dvc push
!git add ../data/processed/train_clean.csv.dvc dvc.yaml
!git commit -m "Versão limpa do dataset treino persistida e versionada com DVC"


 ⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/workspace/.dvc/cache/files/md5'| |0/? [00:00<?,    ?fil
                                                                                
!
  0%|          |Adding ../data/processed/train_clean.c0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /workspace/data/processed0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 12.04file/s]
Pushing
!
  0% Querying remote cache|                          |0/1 [00:00<?,    ?files/s]
100% Querying remote cache|█████████████████████|1/1 [00:00<00:00,  4.31files/s]
                                                                                
!
  0% Querying remote cache|                          |0/0 [00:00<?,    